In [38]:
import os
import getpass
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
class AgentState(TypedDict):
  messages: List[Union[HumanMessage, AIMessage]]

In [40]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

llm = init_chat_model("google_genai:gemini-2.0-flash", temperature=0.9)

In [41]:
def process(state: AgentState):
  response = llm.invoke(state['messages'])
  state['messages'].append(AIMessage(content=response.content))
  print(f"\nAI: {response.content}")

  return state

In [42]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)

agent = graph.compile()

In [43]:
conversation_history = []

while True:
  user_input = input("Enter: ")
  conversation_history.append(HumanMessage(content=user_input))
  if user_input.lower() in ["exit", "quit", "bye"]:
    print("BYE")
    break
  result = agent.invoke({"messages": conversation_history})

conversation_history = result["messages"]


AI: Hello! How can I help you today?

AI: Nice to meet you, Rana! It's a pleasure to connect with you. Is there anything I can help you with today?

AI: As a large language model, I don't experience feelings or emotions like humans do. So, I don't "feel" in the way you might. However, I am functioning well and ready to assist you with any questions or tasks you have. How can I help you, Rana?

AI: I am a large language model, trained by Google. I am designed to be informative and comprehensive. I am trained on a massive amount of text data and am able to communicate and generate human-like text in response to a wide range of prompts and questions. For example, I can provide summaries of factual topics, create stories, and translate languages.

AI: 2 + 8 = 10
BYE


In [ ]:
with open("logging.txt", "w") as file:
  file.write("Your Conversation Log:\n")
  for message in conversation_history:
    if isinstance(message, HumanMessage):
      file.write(f"You: {message.content}\n")
    elif isinstance(message, AIMessage):
      file.write(f"AI: {message.content}\n\n")
  file.write("End Conversation =========")
print("Conversation Saved to loggong.txt")

Conversation Saved to loggong.txt
